In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


StatementMeta(, 8cd6ec35-4c12-451a-a2d9-14faf8d6d2b1, 3, Finished, Available, Finished)

In [2]:
df = spark.read.option("multiline", "true").json("Files/Bing BB.json")
# df now is a Spark DataFrame containing JSON data from "Files/Bing BB.json".
display(df)

StatementMeta(, 8cd6ec35-4c12-451a-a2d9-14faf8d6d2b1, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, dbc2e97b-2888-4419-b22b-90a348ad2320)

In [3]:
from pyspark.sql.functions import explode
df_exploded = df.select(explode("organic_results").alias("result"))
display(df_exploded)

StatementMeta(, 8cd6ec35-4c12-451a-a2d9-14faf8d6d2b1, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 9d9dc888-a887-4d63-9d11-1c05e0a1bccc)

In [4]:
json_list=df_exploded.toJSON().collect()
print(json_list)

StatementMeta(, 8cd6ec35-4c12-451a-a2d9-14faf8d6d2b1, 6, Finished, Available, Finished)

['{"result":{"date":"1h","favicon":"https://www.bing.com/th?id=ODF.9REdhC5rO6fVR7BHF6_cpQ&pid=news&w=16&h=16&c=14&rs=2&qlt=90","link":"https://www.foxweather.com/weather-news/tracking-tropical-cyclone-nine-atlantic-2025","position":1,"snippet":"Tropical storm alerts remain in effect for portions of the eastern Florida coastline, and at least two states have declared preemptive states of emergency as a looming storm system remains on its way ...","source":"FOX Weather","title":"Future Imelda prompts Tropical Storm Watch in Florida as system creeps closer to Southeast US coast"}}', '{"result":{"date":"1h","link":"https://beebom.com/grow-a-garden-prime-seed-pack-guide/","position":2,"snippet":"Check out this guide to get the Prime Seed Pack in Grow a Garden, along with details for all the plants you will find inside.","source":"Beebom","thumbnail":"https://www.bing.com/th?id=OVFT.JHsDVg0Ia8Ne-nYGR3PQwS&pid=News&w=234&h=132&c=14&rs=2&qlt=90","title":"Grow a Garden Prime Seed Pack: All Plan

In [5]:
import json
news_json=json.loads(json_list[0])
print(news_json)

StatementMeta(, 8cd6ec35-4c12-451a-a2d9-14faf8d6d2b1, 7, Finished, Available, Finished)

{'result': {'date': '1h', 'favicon': 'https://www.bing.com/th?id=ODF.9REdhC5rO6fVR7BHF6_cpQ&pid=news&w=16&h=16&c=14&rs=2&qlt=90', 'link': 'https://www.foxweather.com/weather-news/tracking-tropical-cyclone-nine-atlantic-2025', 'position': 1, 'snippet': 'Tropical storm alerts remain in effect for portions of the eastern Florida coastline, and at least two states have declared preemptive states of emergency as a looming storm system remains on its way ...', 'source': 'FOX Weather', 'title': 'Future Imelda prompts Tropical Storm Watch in Florida as system creeps closer to Southeast US coast'}}


In [6]:
print(news_json['result']['source'])
print(news_json['result']['title'])
print(news_json['result']['link'])
print(news_json['result']['snippet'])
print(news_json['result']['date'])

StatementMeta(, 8cd6ec35-4c12-451a-a2d9-14faf8d6d2b1, 8, Finished, Available, Finished)

FOX Weather
Future Imelda prompts Tropical Storm Watch in Florida as system creeps closer to Southeast US coast
https://www.foxweather.com/weather-news/tracking-tropical-cyclone-nine-atlantic-2025
Tropical storm alerts remain in effect for portions of the eastern Florida coastline, and at least two states have declared preemptive states of emergency as a looming storm system remains on its way ...
1h


In [7]:
source=[]
title=[]
link=[]
snippet=[]
date=[]
 
for result_str in json_list:
    try:
        article=json.loads(result_str)
 
        source.append(article['result']['source'])
        title.append(article['result']['title'])
        snippet.append(article['result']['snippet'])
        link.append(article['result']['link'])
        date.append(article['result']['date'])
    except Exception as e:
        print("Errorinf processing JSON object:{e}")

StatementMeta(, 8cd6ec35-4c12-451a-a2d9-14faf8d6d2b1, 9, Finished, Available, Finished)

In [8]:
from pyspark.sql.types import StructType, StructField, StringType
 
# Sample data (make sure source, title, link, date are defined lists of equal length)
data = list(zip(source, title,snippet, link, date))
 
# Define schema with correct field types
schema = StructType([
    StructField("source", StringType(), True),
    StructField("title", StringType(), True),
    StructField("snippet", StringType(), True),
    StructField("link", StringType(), True),
    StructField("date", StringType(), True)
])
 
# Create DataFrame
df_cleaned = spark.createDataFrame(data, schema=schema)
display(df_cleaned)

StatementMeta(, 8cd6ec35-4c12-451a-a2d9-14faf8d6d2b1, 10, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4ab87349-28cf-44b3-a935-76728fb4d4e3)

In [9]:
from pyspark.sql.functions import udf, to_timestamp, year, month, dayofmonth, hour, date_format
from pyspark.sql.types import StringType
import datetime
 
# UDF to convert relative time to formatted datetime string 'dd-MM-yyyy HH:mm:ss'
def simple_convert_date(rel_time):
    now = datetime.datetime.now()
    try:
        if rel_time.endswith('m'):
            dt = now - datetime.timedelta(minutes=int(rel_time[:-1]))
        elif rel_time.endswith('h'):
            dt = now - datetime.timedelta(hours=int(rel_time[:-1]))
        elif rel_time.endswith('d'):
            dt = now - datetime.timedelta(days=int(rel_time[:-1]))
        else:
            return None
        return dt.strftime('%d-%m-%Y %H:%M:%S')
    except:
        return None
 
# Register the UDF
convert_date_udf = udf(simple_convert_date, StringType())
 
# Apply UDF to convert relative time to formatted string
df_with_date = df_cleaned.withColumn("formatted_date", convert_date_udf("date"))
 
# Drop original 'date' column and rename 'formatted_date' to 'date'
df_with_date = df_with_date.drop("date").withColumnRenamed("formatted_date", "date")
 
# Convert the formatted string column to TimestampType
df_with_date = df_with_date.withColumn("date_ts", to_timestamp("date", "dd-MM-yyyy HH:mm:ss"))
display(df_with_date)
 

StatementMeta(, 8cd6ec35-4c12-451a-a2d9-14faf8d6d2b1, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 826a5347-d85f-4747-89d3-4a75cf23b524)

In [10]:
from pyspark.sql.functions import minute, second
 
df_final = df_with_date \
    .withColumn("day", dayofmonth("date_ts")) \
    .withColumn("month", month("date_ts")) \
    .withColumn("year", year("date_ts")) \
     .withColumn("hour", hour("date_ts")) \
     .withColumn("weekday", date_format("date_ts", "EEEE"))
 
df_final = df_final.drop("date_ts")
 
display(df_final)
 

StatementMeta(, 8cd6ec35-4c12-451a-a2d9-14faf8d6d2b1, 12, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c8b5e664-b460-4880-a07a-a60bf8d704a3)

In [11]:
from pyspark.sql.utils import AnalysisException
 
try:
    table_name='Bing_news.latest_new_tbl'
    df_final.write.format("delta").saveAsTable(table_name)
except:
    print("Table Already Exists!")
    df_final.createOrReplaceTempView("vw_df_final")
 
    df_sorted = spark.sql("""
    SELECT * FROM vw_df_final
    ORDER BY date DESC
    """)
 
    spark.sql(f"""
    MERGE INTO Bing_news.latest_new_tbl AS target_table
    USING vw_df_final AS source_view
    ON source_view.link = target_table.link
    WHEN MATCHED AND (
    source_view.source <> target_table.source OR
    source_view.title <> target_table.title OR
    source_view.snippet <> target_table.snippet OR
    source_view.link <> target_table.link OR
    source_view.date = target_table.date OR
    source_view.day <> target_table.day OR
    source_view.month <> target_table.month OR
    source_view.year <> target_table.year OR
    source_view.hour <> target_table.hour OR
    source_view.weekday <> target_table.weekday
    )
 
    THEN UPDATE SET *
   
    WHEN NOT MATCHED THEN INSERT *
    """)

StatementMeta(, 8cd6ec35-4c12-451a-a2d9-14faf8d6d2b1, 13, Finished, Available, Finished)

Table Already Exists!


In [12]:
%%sql
 
SELECT count(*) from Bing_news.latest_new_tbl

StatementMeta(, 8cd6ec35-4c12-451a-a2d9-14faf8d6d2b1, 14, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 1 fields>